<a href="https://colab.research.google.com/github/natdebandi/censo_arg/blob/main/redatam_parser_depto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PARSER simple de archivos de REDATAM

frecuencia simple de una variable a nivel departamento

Natalia Debandi


https://redatam.indec.gob.ar/binarg/RpWebEngine.exe/Portal?BASE=CPV2022&lang=ESP

Para usarlo bajar el archivo al repositorio y BORRAR las primeras lineas del encabezadodel excel.

In [1]:
import pandas as pd
import openpyxl
from typing import List, Dict

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:

def parse_excel_depto(file_path: str, variable_freq: str) -> List[Dict]:
    """
    Parsea un archivo Excel con datos de áreas, departamentos y lugares de residencia.

    Args:
        file_path (str): Ruta al archivo Excel
        variable_freq (str): Nombre de la variable exacta que encontramos en el excel

    Returns:
        List[Dict]: Lista de diccionarios con los datos estructurados
    """
    # Leer el archivo Excel
    df = pd.read_excel(file_path, header=None)

    current_area = None
    current_depto = None
    results = []

    for index, row in df.iterrows():
        # Verificar si es una fila con área y departamento
        #print(row[1])

        if (str(row[1]).startswith('RESUMEN')):
            print('sale')
            break


        if pd.notna(row[1]) and str(row[1]).startswith('AREA #'):
            current_area = row[1]
            current_depto = row[2] if pd.notna(row[2]) else None
            continue

        # Verificar si es el encabezado de la tabla (indica que vienen los datos)
      #  print(row[2])
        if pd.notna(row[1]) and row[1] == variable_freq:
            # Saltar el encabezado y procesar las filas siguientes
            data_rows = []
            next_idx = index + 1

            while next_idx < len(df) and pd.notna(df.iloc[next_idx][1]):
                data_row = df.iloc[next_idx]
                if data_row[2] == 'Total':
                    break

                data_rows.append({
                    'variable_freq': data_row[1],
                    'casos': data_row[2],
                    'porcentaje': data_row[3],
                    'acumulado': data_row[4]
                })
                next_idx += 1

            # Agregar a resultados con el área y departamento actual
            for data in data_rows:
                results.append({
                    'area': current_area,
                    'departamento': current_depto,
                    'variable_freq': data['variable_freq'],
                    'casos': data['casos'],
                    'porcentaje': data['porcentaje'],
                    'acumulado': data['acumulado']
                })

    return results

In [11]:
# Cambiar por la ruta del archivo
file_path = '/content/drive/MyDrive/1. Investigacion/1.working documents/Patagonia/redatam_depto/lugar_nacimiento_depto.xlsx'
# Colocar el nombr exacto que figura en el excel
variable_freq = 'Lugar de nacimiento'

try:
  parsed_data = parse_excel_depto(file_path,variable_freq)
  df_result = pd.DataFrame(parsed_data)
  print(df_result.head())
except Exception as e:
  print(f"Error al procesar el archivo: {e}")


sale
           area departamento             variable_freq   casos  porcentaje  \
0  AREA # 02007     Comuna 1            Esta provincia  110628    0.500577   
1  AREA # 02007     Comuna 1  Otra provincia argentina   56104    0.253863   
2  AREA # 02007     Comuna 1                 Otro país   54269    0.245560   
3  AREA # 02007     Comuna 1                     Total  221001    1.000000   
4  AREA # 02014     Comuna 2            Esta provincia   93683    0.583299   

   acumulado  
0   0.500577  
1   0.754440  
2   1.000000  
3   1.000000  
4   0.583299  


In [12]:
# corrijo el "AREA #"

df_result['area'] = df_result['area'].str.replace('AREA #', '', regex=False)
df_result.head()


,area,departamento,variable_freq,casos,porcentaje,acumulado
0,02007,Comuna 1,Esta provincia,110628,0.500577,0.500577
1,02007,Comuna 1,Otra provincia argentina,56104,0.253863,0.754440
2,02007,Comuna 1,Otro país,54269,0.245560,1.000000
3,02007,Comuna 1,Total,221001,1.000000,1.000000
4,02014,Comuna 2,Esta provincia,93683,0.583299,0.583299


In [7]:
# guardo solo lo que me interesa, no quiero el % ni el acumulado
df_result = df_result[['area', 'departamento', 'variable_freq','casos']]
df_result.head()


,area,departamento,variable_freq,casos
0,02007,Comuna 1,Esta provincia,110628
1,02007,Comuna 1,Otra provincia argentina,56104
2,02007,Comuna 1,Otro país,54269
3,02007,Comuna 1,Total,221001
4,02014,Comuna 2,Esta provincia,93683


In [13]:
# Remove rows where 'variable_freq' is "Total"
df_result = df_result[df_result['variable_freq'] != 'Total']

print(df_result.head())


     area departamento             variable_freq   casos  porcentaje  \
0   02007     Comuna 1            Esta provincia  110628    0.500577   
1   02007     Comuna 1  Otra provincia argentina   56104    0.253863   
2   02007     Comuna 1                 Otro país   54269    0.245560   
4   02014     Comuna 2            Esta provincia   93683    0.583299   
5   02014     Comuna 2  Otra provincia argentina   46733    0.290974   

   acumulado  
0   0.500577  
1   0.754440  
2   1.000000  
4   0.583299  
5   0.874272  


In [15]:
###CAMBIAR EL NOMBRE Y PATH
output_file_path = '/content/drive/MyDrive/1. Investigacion/1.working documents/Patagonia/preprocesados/lugar_nacimiento_depto.csv'
df_result.to_csv(output_file_path, index=False, encoding='utf-8')